In [1]:
import argparse
import logging 
import pandas as pd 
import numpy as np 
!pip install pandasql
import pandasql as ps 
from pathlib import Path
import os, sys 
import logging 



In [2]:
def GetArgs():
        """
        Supports  command-line arguments as the location of source file.
        """
        parser = argparse.ArgumentParser(
           description='Reads 2 arguments  -d (directory name ) -f (file name)')
        parser.add_argument('-d', '--input_dir_name', action='store',default = "c:\\openpay",
                                           help='CSV FILE name.')
        parser.add_argument('-f', '--input_file_name', action='store',default = "sample_data.xlsx",
                                           help='CSV FILE name.')
        
        args = parser.parse_args()
      # print (args)
        return args 
        

    

       
def main():
    v_input_file_name = ""
    v_input_dir_name= ""
    v_args = GetArgs() 
    v_input_dir_name=v_args.input_dir_name
    v_input_file_name=v_args.input_file_name
    return v_input_dir_name, v_input_file_name

In [3]:
def CheckAccesstoFiles(v_source_file):
    """"
      Check Basic Operating System permission for source file 
    """
    try:
        source_file=Path(v_source_file)
        if os.path.isfile(source_file):
            logging.info("Source File available. Continue " + v_source_file)
        else:
            logging.error("Issue opening Source file .Check Location " + v_source_file)
            sys.exit(1)
    except:
        logging.error("I/O error or OS error.Check path  " + v_source_file )
        logging.error(sys.exc_info()[0])
        sys.exit(1)
        
  
        
        
     

In [4]:
def ReadExcel(v_source_file,v_sheetname):
    """
    Read xlsx file . Pass file name and sheet name 
    """
    try:
        df=pd.read_excel(v_source_file,sheet_name=v_sheetname)
        logging.info("Success Loading data Sheet "+v_sheetname+" into dataframe")
        return df 
    except:
        logging.error("Error reading file into dataframe ")
        sys.exit(1)
        
    

In [5]:
def RunSQL(v_sql):
    """
    execute sql and return dataframe using pandassql 
    """
    try:
        df_out = ps.sqldf(v_sql)
        logging.info("Executed SQL successfully " +v_sql )
        return df_out 
        
    except:
        logging.error("Error executing SQL query " +v_sql)
        logging.error(sys.exc_info()[0])
        sys.exit(1)

In [6]:
def WriteCSV(v_dataframe, v_out_file):
    """
    Write datframe to CSV in the working directory 
    """
    try:
        if len(v_dataframe) > 0:
            logging.info("Writing output to CSV File " + v_out_file )
            if (v_dataframe.to_csv(v_out_file, header=True, encoding='utf-8', index=False)):
                logging.info(v_out_file + " Written sucessfully")
    except:
        logging.error("Error Writing output to CSV File for " + v_out_file)
        logging.error(sys.exc_info()[0])
        sys.exit(1)

In [7]:
# MAIN Function 
#Set up logging
v_input_dir_name,v_input_file_name   = main()
v_log_file = "aws_pos.log" 
v_log_file_path=Path(v_input_dir_name +"\\"+ v_log_file)
try:
    os.chdir(v_input_dir_name)
    v_get_dir = os.getcwd()
    print ( "Current working directory " + v_get_dir)
    logging.basicConfig(filename=v_log_file_path, 
                    level=logging.INFO, 
                    filemode='w',
                    format='%(asctime)s - %(name)s - %(threadName)s -  %(levelname)s - %(message)s') 
except IOError:
    print ("Cannot open a file in the directory. Check Directory Permission in " + v_input_dir_name  )
v_input_file_name = "sample_data.xlsx"
v_source_file =  v_input_dir_name + "\\" + v_input_file_name   
CheckAccesstoFiles(v_source_file)



Current working directory c:\openpay


# Read File 



In [8]:
df_Merchant=ReadExcel(v_source_file,'tblMerchant')
df_Merchant.head()

,RetailerID,RetailerName
0,50000000008,RetailerA
1,50000000013,RetailerB
2,50000000014,RetailerC
3,50000000016,RetailerD
4,50000000017,RetailerE


In [9]:
df_Trans=ReadExcel(v_source_file,'tblTrans')
df_Trans.dtypes
df_Trans.head()

,RetailerID,CustomerID,DepositOnUTC,PlanReceivedOnUTC,PurchasePrice,Deposit,RefundedOn,LastUpdated
0,50000000004,50000000140,2019-06-03 06:29:54.210,2019-06-03 06:29:47.993,30.00,7.50,2019-06-03 09:37:03.203,2019-06-03 09:37:05.220
1,50000000004,50000000151,2019-06-03 06:44:54.280,2019-06-03 06:44:47.413,30.00,7.50,2019-06-03 09:40:03.107,2019-06-03 09:40:04.797
2,50000000004,50000000151,2019-06-03 07:12:41.660,2019-06-03 07:12:36.070,30.00,7.61,2019-06-03 09:40:06.250,2019-06-03 09:40:07.250
3,50000000004,50000000152,2019-06-03 07:14:32.323,2019-06-03 07:14:26.107,159.20,39.80,NaT,2019-09-04 01:00:13.930
4,50000000004,50000000154,2019-06-03 07:59:38.033,2019-06-03 07:59:33.133,110.42,27.60,NaT,2019-09-04 01:00:16.293


In [10]:
df_Cust=ReadExcel(v_source_file,'tblCust')
df_Cust.head()

,CustomerID,CreatedOn,LastUpdated,RetailerID
0,50000000128,2019-05-31 11:42:58.610,2019-05-31 11:42:58.610,50000000004
1,50000000130,2019-05-31 12:01:17.133,2019-06-12 08:08:35.790,50000000004
2,50000000130,2019-05-31 12:01:17.133,2019-06-12 08:08:35.790,50000000004
3,50000000151,2019-06-03 06:40:21.370,2019-06-03 06:40:21.370,50000000004
4,50000000151,2019-06-03 06:40:21.370,2019-06-03 06:40:21.370,50000000004


In [11]:
#SQL Data Model 
# Week Starts on Sunday and ends on Sat 
# Assumption : PlanRecievedOnUTC - Date sales has been made 

v_weekly_sales_retailer = """
select 
strftime('%Y', PlanReceivedOnUTC) Year,
strftime('%W', PlanReceivedOnUTC) WeekNumber,
max(date(strftime('%Y-%m-%d',PlanReceivedOnUTC), 'weekday 0', '-7 day')) WeekStart,
max(date(strftime('%Y-%m-%d',PlanReceivedOnUTC), 'weekday 0', '-1 day')) WeekEnd,
RetailerName, 
count(PurchasePrice) as Count_Sales, 
round(sum(PurchasePrice),2) as Total_Sales 
from df_Trans 
inner join df_Merchant  
where df_Trans.RetailerID = df_Merchant.RetailerID
group by Year, WeekNumber, RetailerName
order by Year, WeekNumber, RetailerName 
"""


Weekly_Sales_retailer = RunSQL(v_weekly_sales_retailer)
Weekly_Sales_retailer

,Year,WeekNumber,WeekStart,WeekEnd,RetailerName,Count_Sales,Total_Sales
0,2019,22,2019-06-02,2019-06-08,RetailerM,289,36102.02
1,2019,23,2019-06-09,2019-06-15,RetailerM,337,41514.72
2,2019,24,2019-06-16,2019-06-22,RetailerM,243,29576.59
3,2019,25,2019-06-23,2019-06-29,RetailerM,308,38873.76
4,2019,26,2019-06-30,2019-07-06,RetailerM,329,43888.22
...,...,...,...,...,...,...,...
594,2020,26,2020-06-28,2020-07-04,RetailerS,6,1637.31
595,2020,26,2020-06-28,2020-07-04,RetailerT,5,2642.97
596,2020,26,2020-06-28,2020-07-04,RetailerU,12,659.94
597,2020,26,2020-06-28,2020-07-04,RetailerV,7,2240.58


In [12]:
# WEEKLY REFUNDS BY RETAILER 
# Assumption : Refund Amount = Deposit Amount 
# NULL Data is handled in SQL statement 

v_weekly_refunds_retailer = """
select 
strftime('%Y', RefundedOn) Year,
strftime('%W', RefundedOn) WeekNumber,
max(date(strftime('%Y-%m-%d',RefundedOn), 'weekday 0', '-7 day')) WeekStart,
max(date(strftime('%Y-%m-%d',RefundedOn), 'weekday 0', '-1 day')) WeekEnd,
RetailerName, 
count(Deposit) as Count_Refund, 
round(sum(Deposit),2) as Total_Sales 
from df_Trans 
inner join df_Merchant  
where 
df_Trans.RetailerID = df_Merchant.RetailerID 
and RefundedOn is not null  
group by Year, WeekNumber, RetailerName
order by Year, WeekNumber, RetailerName 
"""

Weekly_Refund_retailer = RunSQL(v_weekly_refunds_retailer)
Weekly_Refund_retailer

,Year,WeekNumber,WeekStart,WeekEnd,RetailerName,Count_Refund,Total_Sales
0,2019,22,2019-06-02,2019-06-08,RetailerM,9,152.32
1,2019,23,2019-06-09,2019-06-15,RetailerM,14,502.11
2,2019,24,2019-06-16,2019-06-22,RetailerM,17,548.84
3,2019,25,2019-06-23,2019-06-29,RetailerM,18,676.89
4,2019,26,2019-06-30,2019-07-06,RetailerM,24,950.48
...,...,...,...,...,...,...,...
461,2020,33,2020-08-16,2020-08-22,RetailerH,9,434.95
462,2020,33,2020-08-16,2020-08-22,RetailerP,23,590.56
463,2020,33,2020-08-16,2020-08-22,RetailerQ,5,200.72
464,2020,33,2020-08-16,2020-08-22,RetailerR,35,1004.70


In [13]:
#Active Customer Per week 
#Active Customer = Atleast 1 plan per week . A customer active multiple times a week will be counted as 1.
#Assumption : LastUpdated column in tbltrans 
# Not taking TblCust Data as the history is lost in noramlised table 

v_active_customer = """
select 
strftime('%Y', LastUpdated) Year,
strftime('%W', LastUpdated) WeekNumber,
max(date(strftime('%Y-%m-%d',LastUpdated), 'weekday 0', '-7 day')) WeekStart,
max(date(strftime('%Y-%m-%d',LastUpdated), 'weekday 0', '-1 day')) WeekEnd,
count(distinct CustomerID) as Active_Customer_Count 
from df_Trans 
group by Year, WeekNumber
order by Year, WeekNumber 

"""

Weekly_Active_Customer = RunSQL(v_active_customer)

Weekly_Active_Customer



,Year,WeekNumber,WeekStart,WeekEnd,Active_Customer_Count
0,2019,22,2019-06-02,2019-06-08,6
1,2019,23,2019-06-09,2019-06-15,14
2,2019,24,2019-06-16,2019-06-22,13
3,2019,25,2019-06-23,2019-06-29,16
4,2019,26,2019-06-30,2019-07-06,20
...,...,...,...,...,...
60,2020,29,2020-07-19,2020-07-25,9905
61,2020,30,2020-07-26,2020-08-01,20486
62,2020,31,2020-08-02,2020-08-08,14470
63,2020,32,2020-08-09,2020-08-15,15372


# Write Ouput to file 

In [14]:
WriteCSV(Weekly_Sales_retailer , 'WeeklySalesRetailer.csv')
WriteCSV(Weekly_Refund_retailer , 'WeeklyRefundRetailer.csv')
WriteCSV(Weekly_Active_Customer , 'WeeklyActiveCustomer.csv')